In [1]:
%pip install opencv-python mediapipe pandas scikit-learn tensorflow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import cv2
import mediapipe as mp
import pandas as pd

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
kategori = ['pukulan lurus', 'pukulan suwing']  # sesuaikan dengan folder
data_all = []

for label in kategori:
    folder = f'dataset/pukulan/{label}'
    for filename in os.listdir(folder):
        if filename.endswith('.mp4'):
            cap = cv2.VideoCapture(os.path.join(folder, filename))
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(image)
                if results.pose_landmarks:
                    row = []
                    for lm in results.pose_landmarks.landmark:
                        row.extend([lm.x, lm.y, lm.z, lm.visibility])
                    row.append(label)
                    data_all.append(row)
            cap.release()

pose.close()

# Simpan ke DataFrame
num_landmarks = 33 * 4  # x, y, z, v
columns = [f'{name}_{axis}' for name in range(33) for axis in ['x', 'y', 'z', 'v']]
columns.append('label')

df = pd.DataFrame(data_all, columns=columns)
df.to_csv('csv/pose_dataset_pukulan2.csv', index=False)
print("✅ Dataset disimpan ke pose_dataset_pukulan2.csv")


✅ Dataset disimpan ke pose_dataset_pukulan2.csv


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ✅ 1. Baca dataset
df = pd.read_csv('csv/pose_dataset_pukulan2.csv')

# ✅ 2. Ambil hanya titik-titik tangan (6 keypoint × 4 = 24 fitur)
selected_indices = [11, 12, 13, 14, 15, 16]
selected_columns = []

for i in selected_indices:
    for axis in ['x', 'y', 'z', 'v']:
        selected_columns.append(f"{i}_{axis}")

X = df[selected_columns].values.astype('float32')
y = LabelEncoder().fit_transform(df['label'])
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# ✅ 3. Buat dan latih model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(y.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stop])

model.save("model/model_pukulan2.h5")
print("✅ Model disimpan ke model_pukulan2.h5")


Epoch 1/50


c:\Users\filam\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5774 - loss: 0.6821 - val_accuracy: 0.6047 - val_loss: 0.6547
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6173 - loss: 0.6524 - val_accuracy: 0.6304 - val_loss: 0.6128
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6809 - loss: 0.6034 - val_accuracy: 0.7989 - val_loss: 0.4926
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7609 - loss: 0.5078 - val_accuracy: 0.8568 - val_loss: 0.3981
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8098 - loss: 0.4301 - val_accuracy: 0.8981 - val_loss: 0.3079
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8484 - loss: 0.3646 - val_accuracy: 0.9186 - val_loss: 0.2640
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8680 - loss: 0.3284 - val_accuracy: 0.9337 - val_loss: 0.2127
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8949 - loss: 0.2778 - val_accuracy: 0.9492 - val_

✅ Model disimpan ke model_pukulan2.h5


In [6]:
import tensorflow as tf
from tensorflow import keras

# Load model dari .h5
model = keras.models.load_model("model/model_pukulan2.h5")

# Konversi ke TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan ke file
with open("model/model_pukulan2.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model TFLite disimpan ke model_pukulan2.tflite")


INFO:tensorflow:Assets written to: C:\Users\filam\AppData\Local\Temp\tmpc74h9dyf\assets


INFO:tensorflow:Assets written to: C:\Users\filam\AppData\Local\Temp\tmpc74h9dyf\assets


Saved artifact at 'C:\Users\filam\AppData\Local\Temp\tmpc74h9dyf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 24), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  1954476549776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1954476549200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1954476548048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1954476540944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1954476548432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1954476555536: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Model TFLite disimpan ke model_pukulan2.tflite
